<a href="https://colab.research.google.com/github/Guidong99/statistic-optimization/blob/main/NEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from numpy import transpose as t
from numpy import array as array
import numpy as np
import pandas as pd
from scipy.optimize import minimize,Bounds,LinearConstraint
from scipy.special import gamma
from math import exp,pi
import matplotlib.pyplot as plt
from scipy.stats import norm
def gaussian(x,mu,sigma2):
  return 1/(2*pi*sigma2)*exp(-(x-mu)**2/2*sigma2)
def chi_square(x,k):
  return (x**(k/2-1)*exp(-x/2))/(2**(k/2)*gamma(k/2)) if x>0 else 0
def generate_C(x,s2,a,b):
  m,K,S=len(x),len(a),len(b)
  C=[]
  c=[[0]*S for i in range(K)]
  for r in range(m):
    for i in range(K):
      for j in range(S):
        c[i][j]=gaussian(x[r],a[i],b[j])*chi_square((n-1)*s2[r]/b[j],n-1)
    C.append(array(c))
  return C



#NEB(Nonelinear Empirical Bayes) class
### 1.arguments:
x
s2
a
b
### 2.functions:
objection function

gradient

hessian



In [256]:
#@title
class NEB():
  n=20
  C=None
  print(len('1'))
  def _init_(self,x,s2,a,b):
    self.x=x
    self.s2=s2
    self.a=a
    self.b=b

  def obj(self):
    k=len(self.a)
    # C=self.C
    def obj(x):
      res=1
      for r in range(len(C)):
        res=res*t(x[:k])@C[r]@x[k:]
      return res
    return obj
  
  def gradient(self):
    # C=self.C
    K=len(self.a)
    def l_gradient(g,h,r,i,tp):
      if tp=='g':
        res=(C[r][i]@h)/(t(g)@C[r]@h)
      if tp=='h':
        res=(t(g)@C[r][:,i])/(t(g)@C[r]@h)
      return res
    def gradient(x):
      g,h=x[:K],x[K:]
      res=[]
      for i in range(len(x)):
        if i<K:
          s=sum(l_gradient(g,h,r,i,'g') for r in range(len(C)))
        else:
          s=sum(l_gradient(g,h,r,i-K,'h') for r in range(len(C)))
        res.append(s)
      return res
    return gradient

  def Hess(self):
    K=len(self.a)
    # C=self.C
    def Hess(x):
      hess=[[0]*m for i in range(m)]
      g,h=x[:K],x[K:]
      def H11(i,j):
        return -sum((t(h)@C[r][i])*(C[r][j]@h)/(t(g)@C[r]@h)**2 for r in range(len(C)))
      def H22(i,j):
        return -sum(t(g)@C[r][:,i]*t(C[r][:,j])@g/(t(g)@C[r]@h)**2 for r in range(len(C)))
      def H12(i,j):
        return sum(C[r][i][j]/(t(g)@C[r]@h)-t(g)@C[r][:,j]*C[r][i]@h/(t(g)@C[r]@h)**2 for r in range(len(C)))
      def H21(i,j):
        return sum(C[r][j][i]/(t(g)@C[r]@h)-t(g)@C[r][:,i]*C[r][j]@h/(t(g)@C[r]@h)**2 for r in range(len(C)))
      for i in range(m):
        for j in range(m):
          if i<K:
            if j<K: hess[i][j]=H11(i,j)
            else: hess[i][j]=H12(i,j-K)
          else:
            if j<K: hess[i][j]=H21(i-K,j)
            else: hess[i][j]=H22(i-K,j-K)
      return hess
    return Hess
  def constraints(self):
    constraint=[]
    K,S=len(self.a),len(self.b)
    constraint.append([1]*K+[0]*S)
    constraint.append([0]*K+[1]*S)
    bound0,bound1=[0]*(K+S),[np.inf]*(K+S)
    return constraint,bound0,bound1


    



  


1


#Example

In [257]:
m=50
n=20
x=np.random.normal(0,1,m)
s2=np.random.chisquare(n-1,m)

a=np.linspace(min(x),max(x),20)
b=np.linspace(min(s2),max(s2),30)

g=[1]*len(a)
h=[1]*len(b)
x=array(g+h)

s=NEB()
s.x=x
s.s2=s2
s.a=a
s.b=b
s.C=generate_C(x,s2,a,b)

Obj=s.obj()
Grad=s.gradient()
Hess=s.Hess()

In [258]:
Obj(x),Grad(x),Hess(x)
constraints,bound0,bound1=s.constraints()

#Tools Camparing

In [262]:
# bound1

linear_constraint=LinearConstraint(constraints,[1,1],[1,1])
bounds=Bounds(bound0,bound1)

In [263]:
res = minimize(Obj, x, method='SLSQP', jac=Grad, hess=Hess,
               constraints=linear_constraint,
               options={'ftol': 1e-9, 'disp': True}, bounds=bounds)
res.x

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0
            Iterations: 1
            Function evaluations: 2
            Gradient evaluations: 1


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:511: RuntimeWarning: Method SLSQP does not use Hessian information (hess).
  RuntimeWarning)


array([7.92196536e-02, 7.92196536e-02, 7.92196536e-02, 7.92196536e-02,
       7.92196484e-02, 7.92191527e-02, 7.91911909e-02, 7.82276017e-02,
       5.70407833e-02, 4.37982983e-13, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.35700348e-13, 4.90163465e-13,
       7.28096036e-02, 7.89798201e-02, 7.92140114e-02, 7.92195734e-02,
       1.25222664e-01, 9.34807787e-14, 5.49560397e-14, 7.77156117e-15,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.16573418e-14,
       3.14193116e-14, 4.60742555e-14, 6.81676937e-14, 8.85957974e-14,
       1.05138120e-13, 2.85608907e-02, 1.05864312e-01, 1.79075392e-01,
       2.48153795e-01, 3.13122946e-01])